# Laboratorio Modulo C - scikit learn - Unsupervised Learning

# Clustering


In [ ]:
from sklearn import datasets
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import random

Importo kmeans

In [ ]:
from sklearn.cluster import KMeans

Posso analizzare la bontà del clustering ottenuto calcolando il silhouette score: https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html

In [ ]:
from sklearn.metrics import silhouette_score

Esempio didattico: dati sintetici.

In [ ]:
n_samples = 50
delta_1 = 1
delta_2 = 3
delta_3 = 5
delta_4 = 7

X_synth = []

for i in range(n_samples):
    X_synth.append((random.random() + delta_1, random.random() + delta_1))
    X_synth.append((random.random() + delta_2, random.random() + delta_2))
    X_synth.append((random.random() + delta_3, random.random() + delta_3))
    X_synth.append((random.random() + delta_4, random.random() + delta_4))


X_plot = [Xs for Xs, Ys in X_synth]
Y_plot = [Ys for Xs, Ys in X_synth]

# seleziono kmeans con 4 cluster
n_clusters = 4

estimator = KMeans(n_clusters=n_clusters)
estimator.fit(X_synth)
score = silhouette_score(X_synth, estimator.labels_)

print(f"Silhouette score: {score}")
print(f"Etichette per ogni punto: {estimator.labels_}")
print(f"Centroidi: {estimator.cluster_centers_}")

plt.figure()
plt.scatter(X_plot, Y_plot, color="cornflowerblue", label=f"n_clusters={n_clusters}")

for c in estimator.cluster_centers_:
    plt.scatter(c[0], c[1], color="red")

plt.xlabel("X")
plt.ylabel("Y")
plt.title("Example")
plt.legend()
plt.show()



**Domanda:** graficare l'andamento dei centri dei cluster aumentando il numero di cluster. Come varia il silohuette score?

Vediamo ora il clustering applicato sul dataset IRIS https://scikit-learn.org/stable/auto_examples/cluster/plot_cluster_iris.html#sphx-glr-auto-examples-cluster-plot-cluster-iris-py

In [ ]:
np.random.seed(5)
iris = datasets.load_iris()
X = iris.data

In [ ]:
# seleziono kmeans con 5 cluster
n_clusters = 5
estimator = KMeans(n_clusters=n_clusters)

# fit
estimator.fit(X)

Mostro i risultati

In [ ]:
estimators = [
    ("k_means_iris_8", KMeans(n_clusters=8))
]

# reimposto la colormap
# plt.set_cmap('jet')

# il seguente codice serve per mostrare i grafici
fignum = 1
kmeans_labels = []
kmeans_cluster_centers = []
for name, est in estimators:
    fig = plt.figure(fignum, figsize=(4, 3))
    ax = fig.add_subplot(111, projection="3d", elev=48, azim=134)
    ax.set_position([0, 0, 0.95, 1])
    est.fit(X)
    labels = est.labels_
    kmeans_labels.append(labels)
    kmeans_cluster_centers.append(est.cluster_centers_)

    ax.scatter(X[:, 3], X[:, 0], X[:, 2], c=labels.astype(float), edgecolor="k")

    ax.w_xaxis.set_ticklabels([])
    ax.w_yaxis.set_ticklabels([])
    ax.w_zaxis.set_ticklabels([])
    ax.set_xlabel("Petal width")
    ax.set_ylabel("Sepal length")
    ax.set_zlabel("Petal length")
    ax.set_title(name)
    ax.dist = 12
    fignum = fignum + 1


fig.show()

**Domanda**: quali parametri posso modificare per controllare il comportamento dell'algoritmo? Provare ad aggiungere degli elementi nella lista `estimators` così da graficarli.

In [ ]:
print(silhouette_score(X, kmeans_labels[0]))

**Esercizio:** modificare il codice precedente per stampare il silhouette score per clustering eseguito con numero di cluster che varia da 2 a 10.

In [ ]:
for nc in range(2,9):
    est = KMeans(n_clusters=nc)
    est.fit(X_synth)
    labels = est.labels_
    score = silhouette_score(X_synth, est.labels_)
    print(score)

# Regole associative

Vediamo ora come utilizzare le regole associative e l'algoritmo apriori.
Ci serve un'ulteriore dipendenza: `mlxtend` http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/

In [ ]:
!pip install mlxtend

Partiamo dal seguente dataset

In [ ]:
dataset = [['Latte', 'Cipolla', 'Banana', 'Fagioli', 'Uova', 'Yogurt'],
           ['Kiwi', 'Cipolla', 'Banana', 'Fagioli', 'Uova', 'Yogurt'],
           ['Latte', 'Mela', 'Fagioli', 'Uova'],
           ['Latte', 'Pera', 'Mais', 'Fagioli', 'Yogurt'],
           ['Mais', 'Cipolla', 'Cipolla', 'Fagioli', 'Gelato', 'Uova']]

La libreria richiede il processamento dei dati

In [ ]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

Calcoliamo quali itemset hanno almeno supporto 60%.

In [ ]:
from mlxtend.frequent_patterns import apriori

apriori(df, min_support=0.6, use_colnames=True)

Posso combinare supporto e lunghezza delle regole

In [ ]:
frequent_itemsets = apriori(df, min_support=0.6, use_colnames=True)
# aggiungo la lunghezza
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

In [ ]:
frequent_itemsets[ (frequent_itemsets['length'] == 3) &
                   (frequent_itemsets['support'] >= 0.5) ]